In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', None)    # None means no limit
pd.set_option('display.max_columns', None)    # None means no limit
from functions_variables import *
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.dates as mdates
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import pickle
from xgboost import XGBClassifier
from ydata_profiling import ProfileReport
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance


In [ ]:
# Load data
df = pd.read_csv('../data/preprocessed/macro_finance_data_2014_2023.csv')
df.tail()

### Create Target Variable for TSX Index if Index has Increased from the Prior Day

In [ ]:
df['Index Up'] = df['TSX'] > df['TSX'].shift(1)
df.head(10)

## For EDA set the date as the index to leverage time series specific methods for understanding the data

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [ ]:
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['DayofWeek'] = df.index.dayofweek
df.tail()

In [ ]:
with open('../data/models/baseline2.pkl', 'wb') as file:
    pickle.dump(df, file)

In [ ]:
# create a dataframe that only includes the weekdays
# commodities and currency are traded on weekend, but the index is only traded during the week, therefore the number of False values will be present 
# if the weekends are included and distort the percentage of index up calculations
df = df[df['DayofWeek'] < 5]
df.tail(10)

In [ ]:
#group index up by month
grouped = df.groupby(['Year','Month'])['Index Up'].sum()
print(grouped)

In [ ]:
#calculate the number of trading days per month
trading_days = df.groupby(['Year', 'Month']).size()

# Calculate the percentage of Index Up for each month relative to the total
percentage_index_up = (grouped / trading_days) * 100

# Combine the results into a DataFrame
result = pd.DataFrame({
    'Index Up Sum': grouped,
    'Total Trading Days': trading_days,
    'Percentage of Index Up': percentage_index_up
})
result

In [ ]:
result.describe()

In [ ]:
# Calculate the average percentage of index up per trading day for each month for all 10 years combined
month_grouped = result.groupby('Month')['Percentage of Index Up'].mean()
month_grouped

In [ ]:
month_grouped.mean()

In [ ]:
# Calculate the average percentage of index up per trading day for each year for all 10 years
year_grouped = result.groupby('Year')['Percentage of Index Up'].mean()
year_grouped

In [ ]:
year_grouped.mean()

In [ ]:
# To determine if there are any specifc days on which the stock index goes up
day_grouped = df.groupby('Day')['Index Up'].sum()
day_grouped.sort_values(ascending=False)

### Date-Time Components Conclusion
<p>The analysis didn't provide any meaningful patterns or trends involved. The index increased slightly more on a daily basis than going down at an average of 52.4% of the time. The maximum month where the index went up was 82.6% of the time in 2019 and the minimum was 2017 September at 27%. During the Covid Pandemic the market actually performed better in 2019-2021 with the index increasing greater than 55% of the time. Monthly fluctions occured, but on average each year the results are similar across all 10 years. It appears that the middle of the month has higher than normal times where the index went up, with the number of days being the highest for those days.</p>

## Time Series Graphing

In [ ]:
column_list = df.columns[:-5]
column_list

In [ ]:
for column in column_list:
    plot_line_chart(df.index,df[column], column)

## Correlation

In [ ]:
# Correlation for all features
correlation_matrix = df.corr()
correlation_matrix

In [ ]:
# Correlation with just the target 
correlation_matrix['TSX']

In [ ]:
plt.figure(figsize=(20, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, linewidths=0.5)
plt.show()


In [ ]:
# Identify upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]

# Drop those features with correlation greater than 0.9
# Although TSX has correlation greater than 0.9 it will not be dropped
to_drop.remove('EMA')
to_drop.remove('TSX')
df_reduced = df.drop(columns=to_drop)

# Print the results
print(f"Features dropped: {to_drop}")
print(f"Remaining features: {df_reduced.columns.tolist()}")
print(f"Number of features remaining: {df_reduced.shape[1]}")

In [ ]:
# Correlation for all features
correlation_matrix = df.corr()
correlation_matrix

# Identify upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]

# Drop those features with correlation greater than 0.9
# Although TSX has correlation greater than 0.9 it will not be dropped
to_drop.remove('EMA')
to_drop.remove('TSX')
df_reduced = df.drop(columns=to_drop)

# Print the results
print(f"Features dropped: {to_drop}")
print(f"Remaining features: {df_reduced.columns.tolist()}")
print(f"Number of features remaining: {df_reduced.shape[1]}")

## IMPORT BASELINE DATAFRAME

In [ ]:
with open('../data/models/baseline2.pkl', 'rb') as file:
    df = pickle.load(file)

In [ ]:
df.head()

# Feature Engineering

## Time-based Features: Exponential Moving Average
<p>An Exponential Moving Average (EMA) is a type of moving average that places a greater weight and significance on the most recent data points. It is commonly used in time series analysis, especially in financial markets, to smooth out data and identify trends more effectively than a simple moving average (SMA).</p>

In [ ]:
n = 20 # period for the EMA. Used to track the trend of a stock over approximately one month, which is inline with other economic features.

df['EMA'] = df['TSX'].ewm(span=n, adjust=False).mean()
df.head()

In [ ]:
# Computing the difference between consecutive EMA values, which essentially give the slope of the EMA
df['EMA Slope'] = df['EMA'].diff()
df.head()

In [ ]:
# EMA divided by the closing price of the TSX index
df['EMA/Close'] = df['EMA'] / df['TSX']
df.head()

In [ ]:
# The difference between the EMA and the TSX index. If TSX is greater than moving average value will be positive.
df['EMA Divergence'] = df['TSX'] - df['EMA']
df.head()

## Technical Indicators: Relative Strength Index (RSI)
<p>The Relative Strength Index (RSI) is a momentum oscillator used in technical analysis that measures the speed and change of price movements. It is typically used to identify overbought or oversold conditions in a market. The RSI oscillates between 0 and 100, making it easy to interpret and apply in various trading strategies.</p>
<p>Below 30: The asset is generally considered oversold, indicating that it may be undervalued and due for a rebound. </p>
<p>Above 70: The asset is generally considered overbought, indicating that it may be overvalued and due for a correction or pullback.</p>

In [ ]:
df['RSI'] = calculate_rsi(df['TSX'])

## Technical Indicators: Moving Average Convergence Divergence (MACD)

In [ ]:
# Calculate the MACD line
df['MACD'] = df['TSX'].ewm(span=12, adjust=False).mean() - df['TSX'].ewm(span=26, adjust=False).mean()

In [ ]:
# Calculate the Signal line (9-day EMA of MACD) 
df['Signal Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

In [ ]:
# Calculate the MACD Histogram 
df['MACD Histogram'] = df['MACD'] - df['Signal Line']

In [ ]:
# Drop the unnecessary MACD line and Signal line columns from features
df = df.drop(columns=['MACD', 'Signal Line'])

## Price Transformations: Daily Returns and Volatility
<p>Understanding the price fluctuations of a financial asset and measuring the risk associated with the asset on a daily basis.</p>

In [ ]:
# Calculate Daily Returns
df['Daily Return'] = df['TSX'].pct_change()  

# Calculate the Rolling Standard Deviation of Daily Returns
df['Daily Volatility'] = df['Daily Return'].rolling(window=14).std()

df = df.drop(columns=['Daily Return']) # Simple returns is the same as the target variable, positive return equals index up, therefore will not include

In [ ]:
print(df[['TSX', 'Daily Volatility']].tail())

In [ ]:
# Fill all NaN values with zero for EMA, RSI and Volatility
df = df.fillna(0)

In [ ]:
df.head()

In [ ]:
df.info()

Remove correlation once EDA complete

In [ ]:
# Correlation for all features
correlation_matrix = df.corr()
correlation_matrix

# Identify upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.9)]

# Drop those features with correlation greater than 0.9
# Although TSX has correlation greater than 0.9 it will not be dropped
#to_drop.remove('EMA')
to_drop.remove('TSX')
df_reduced = df.drop(columns=to_drop)

# Print the results
print(f"Features dropped: {to_drop}")
print(f"Remaining features: {df_reduced.columns.tolist()}")
print(f"Number of features remaining: {df_reduced.shape[1]}")

### Generate EDA Report

In [ ]:
# Generate the profiling report with time-series mode enabled and sorted by variable type
profile = ProfileReport(df, tsmode=True, title="Time-Series Data Profiling Report")

# Save the report to an HTML file
profile.to_file("your_report.html")

In [ ]:
df_reduced.info()

## Time-Series-Split Data for Time Series Data Set

In [ ]:
X = df_reduced.drop(columns=['Index Up'])
y = df_reduced['Index Up']

In [ ]:
tss = TimeSeriesSplit(n_splits=3)

In [ ]:
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
scaler = StandardScaler()

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns) # maintain column names
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [ ]:
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [ ]:
X_train.to_csv('../data/final/X_train.csv', index=False)
X_test.to_csv('../data/final/X_test.csv', index=False)

X_train_scaled.to_csv('../data/final/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/final/X_test_scaled.csv', index=False)
y_train.to_csv('../data/final/y_train.csv', index=False)
y_test.to_csv('../data/final/y_test.csv', index=False)

In [ ]:
with open('../data/final/scalar.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Recursive Feature Elimination - Logistic Regression Model

In [ ]:
# Initialize Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)

# Initialize RFE with Logistic Regression and the number of features to select
rfe = RFE(estimator=log_reg, n_features_to_select=25)

# Fit RFE on the scaled training data
rfe.fit(X_train_scaled, y_train)

# Get the mask of selected features
selected_features = rfe.support_

# Convert boolean mask to the list of selected feature names
selected_feature_names = X_train.columns[selected_features]

# Filter the training and test data with the selected features
X_train_selected = X_train[selected_feature_names]
X_test_selected = X_test[selected_feature_names]

# Fit Logistic Regression on the selected features
log_reg.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test_selected)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multiclass

# Display results
print("Selected Features:", selected_feature_names.tolist())
print("F1 Score:", f1)

Number of features (F1 Score):

17 - 0.644

20 - 0.656

25 - 0.636

## Recursive Feature Elimination - XGBoost Model

In [ ]:
model = XGBClassifier(random_state=42, eval_metric='mlogloss')

# Create an RFE object, specifying the model and the number of features to select
rfe = RFE(estimator=model, n_features_to_select=20)

# Fit the RFE model on the data
rfe.fit(X_train_scaled, y_train)

# Get the rankings of the features
ranking = rfe.ranking_
selected_features = X_train_scaled.columns[rfe.support_]

print(f"Feature Rankings: {ranking}")
print(f"Selected Features: {selected_features.tolist()}")

X_train_rfe = rfe.transform(X_train)
X_test_rfe = rfe.transform(X_test)

# Train a model on the selected features
model.fit(X_train_rfe, y_train)
y_pred = model.predict(X_test_rfe)
f1 = f1_score(y_test, y_pred, average='binary')
print("Model F1 Score with Selected Features:", f1)

The XGBoost RFE technique yielded the best results so far with an F1 score of 0.718, which is better than the previous benchmark of 0.71. Using the selected features above, we will see how the hyper paramater tuning will help improve the model performance.

Number of features (F1 Score):

17 - 0.708

20 - 0.713

25 - 0.684



## Feature Importance Using Random Forest Classifier

In [ ]:

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train_scaled, y_train)

# Get feature importances
feature_importances = rf_model.feature_importances_

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': X_train_scaled.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Plotting feature importances
plt.figure(figsize=(10, 10))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance using Random Forest')
plt.gca().invert_yaxis() # To display the most important feature at the top
plt.show()

There are several features that seem to be unimportant with importance values below the others using the Random Forest model feature importance selection, but I decided to test eliminating features below the DayofWeek to see how the model performs. After removing the below features I ran the dataset through all the models again , but the models didn't improve above the benchmark of 0.73 from Logistic Regression, but had a F1 Score of 0.71 from the Logistic Regression model.

In [ ]:
X_train_scaled = X_train_scaled.drop(['Month', 'GDP', 'Housing starts','Interest_Rate', 'Unemployment', 'CPI'], axis=1)

In [ ]:
X_test_scaled = X_test_scaled.drop(['Month', 'GDP', 'Housing starts','Interest_Rate', 'Unemployment', 'CPI'], axis=1)

In [ ]:
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

## Feature Importance Using XGBoost Classifier

In [ ]:
# Initialize XGBClassifier
xgb_model = xgb.XGBClassifier(eval_metric='logloss')

# Fit the model to the training data
xgb_model.fit(X_train_scaled, y_train)

# Get feature importances using the 'Gain' method
feature_importances = xgb_model.feature_importances_

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'Feature': X_train_scaled.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Print the DataFrame
print(feature_importance_df)

# Visualize feature importances using XGBoost's built-in plot_importance
plt.figure(figsize=(10, 8))
plot_importance(xgb_model, importance_type='gain')  # Choose from 'weight', 'gain', 'cover'
plt.title('Feature Importance by Gain')
plt.show()

There are no clear features that are unimportant with importance values much below the others using the XGBoost model feature importance selection, but I decided to test eliminating features below the TSX importance value of 1.2 to see how the model performs. After removing the below features and running the dataset through all the models againg the Logistic Regression model improved above the benchmark of 0.73 to an F1 Score of 0.74.

In [ ]:
X_train_scaled = X_train_scaled.drop(['DayofWeek','DAX','Lumber Price', 'GBPCAD', 'CADJPY'], axis=1)

In [ ]:
X_test_scaled = X_test_scaled.drop(['DayofWeek','DAX','Lumber Price', 'GBPCAD', 'CADJPY'], axis=1)

In [ ]:
X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [ ]:
X_test_scaled.columns